# REST Endpoints 기반 A/B Test 수행

“production variants”이라는 개념으로 하나의 SageMaker Endpoint 내에서 여러 신규 모델을 테스트하고 배포 할 수 있습니다. 이러한 variants은 하드웨어 (CPU/GPU), 데이터(코미디/영화), 지역(US West 또는 AP South)에 따라 다를 수 있습니다. Canary 배포와 블루/그린 배포를 위해 endpoint의 모델 간에 트래픽을 전환 할 수 있습니다. A/B 테스트를 위해 트래픽을 분할 할 수 있습니다. 또한 초당 requests와 같이 특정 메트릭을 기반으로 endpoints를 scale-out 또는 in이 가능합니다. 더 많은 request이 들어 오면 SageMaker는 자동으로 model prediction API를 확장합니다.
#### scale-out/in 기능은 Endpoint가 T 시리즈 인스턴스를 사용하는 경우 disable 됩니다.

<img src="img/model_ab.png" width="80%" align="left">

실제 production 환경에서 서로 다른 모델을 비교하고 테스트하기 위해 트래픽 분할을 사용하여 사용자의 subset에 다른 모델 variants로 지정할 수 있습니다. 목표는 어떤 variants가 더 좋은 성능을 가지고 있는지 확인하는 것입니다. 이러한 테스트는 통계적으로 유의미하게 수행하기 위해 오랜 기간(주) 동안 실행해야하는 경우가 종종 있습니다. 위 그림에서 2개의 variants 사이에 임의의 50-50 트래픽 분할을 사용하여 배포 된 2개의 추천 모델을 보여줍니다.

In [ ]:
import boto3
import time
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

### 이전 Notebook에서 Endpoints를 생성하신 경우에는 Endpoints를 삭제하신 후 이 노트북을 생성하시기 바랍니다. 그렇지 않은 경우 HOL를 수행하시는 동안에 ResourceLimitExceeded error를 보실 수 있습니다.

In [ ]:
list_endpoints = sm.list_endpoints()

for ep in list_endpoints['Endpoints']:
    sm.delete_endpoint(EndpointName=ep['EndpointName'])
    

NextToken = 'None'
while NextToken !='':
    lec = sm.list_endpoint_configs(NextToken=NextToken) if NextToken != 'None' else sm.list_endpoint_configs()
    for epc in lec['EndpointConfigs']:
        print(epc['EndpointConfigName'])
        sm.delete_endpoint_config(EndpointConfigName=epc['EndpointConfigName'])
        time.sleep(3)
    NextToken = lec['NextToken'] if lec.get('NextToken') else ''

NextToken = 'None'
while NextToken !='':
    lec = sm.list_models(NextToken=NextToken) if NextToken != 'None' else sm.list_models()
    for epc in lec['Models']:
        print(epc['ModelName'])
        sm.delete_model(ModelName=epc['ModelName'])
        time.sleep(3)
    NextToken = lec['NextToken'] if lec.get('NextToken') else ''

In [ ]:
%store -r

In [ ]:
print(training_job_name)

# Notebook 내 Model 복사

In [ ]:
models_dir = './models'

In [ ]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz $models_dir/model.tar.gz

In [ ]:
import tarfile
import pickle as pkl

#!ls -al ./models

tar = tarfile.open('{}/model.tar.gz'.format(models_dir))
tar.extractall(path=models_dir)
tar.close()

# Prediction Signature 살펴보기
CLI 를 통해 모델의 Input/output을 조회할 수 있습니다.

In [ ]:
!saved_model_cli show --all --dir $models_dir/tensorflow/saved_model/0/

In [ ]:
import boto3
client = boto3.client("sagemaker")

# Training job에서 Model A의 variant 생성

Notes:

* Training과 Inference 이미지가 다르기 때문에 `primary_container_image`가 필요합니다.
* 기본적으로 Training 이미지가 사용되므로, 추가적으로 재정의가 필요합니다.
* https://github.com/aws/sagemaker-python-sdk/issues/1379 참조
* 이 variant는 Elastic Inference를 사용하므로 Elastic Inference 이미지가 필요합니다

In [ ]:
import time
timestamp = '{}'.format(int(time.time()))

model_a_name = '{}-{}-{}'.format(training_job_name, 'var-a', timestamp)

sess.create_model_from_job(name=model_a_name,
                           training_job_name=training_job_name,
                           role=role,
                           image_uri='763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-inference-eia:2.0.0-cpu-py36-ubuntu18.04'.format(region))


# Training job에서 Model B의 variant 생성
Notes:
* 이 모델은 Variant A와 동일하지만, EIA를 사용하지 않습니다.

In [ ]:
model_b_name = '{}-{}-{}'.format(training_job_name, 'var-b', timestamp)

sess.create_model_from_job(name=model_b_name,
                           training_job_name=training_job_name,
                           role=role,
                           image_uri='763104351884.dkr.ecr.{}.amazonaws.com/tensorflow-inference:2.0.0-cpu-py36-ubuntu18.04'.format(region))


# Canary Rollouts and A/B Testing

Canary rollouts은 5% 정도의 사용자에게만 신규 모델을 안전하게 제공하는데 활용됩니다. 전체 사용자 기반에 영향을 주지 않고 실제 production에서 테스트를 하려는 경우에 유용합니다. 대부분의 트래픽은 기존 모델로 이동하므로 Canary 모델의 클러스터 크기는 트래픽이 5%에 불과하기 때문에 상대적으로 작을 수 있습니다.

`deploy()`함수를 사용하는 대신에 Canary 배포와 A/B 테스팅에 대한 다중 variants로 `Endpoint Configuration`를 생성합니다.

In [ ]:
import time
timestamp = '{}'.format(int(time.time()))

endpoint_config_name = '{}-{}'.format(training_job_name, timestamp)

endpoint_config = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
         'VariantName': 'VariantA',
         'ModelName': model_a_name,
         'InstanceType':'ml.m5.large',
         'InitialInstanceCount': 1,
         'InitialVariantWeight': 50,
        'AcceleratorType':'ml.eia2.medium' # This variant will use an Elastic Inference Adapter (GPU)            
        },
        {
         'VariantName': 'VariantB',
         'ModelName': model_b_name,
         'InstanceType':'ml.m5.large',
         'InitialInstanceCount': 1,
         'InitialVariantWeight': 50,
        }
    ])

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpointConfig/{}">REST Endpoint Configuration</a></b>'.format(region, endpoint_config_name)))


In [ ]:
endpoint_name = '{}-{}'.format(training_job_name, timestamp)

endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint</a></b>'.format(region, endpoint_name)))


<h2><span style="color:red">위 Endpoint가 Deploy되기 전까지 기다려 주시기 바랍니다.</span></h2>

In [ ]:
client = boto3.client('sagemaker')
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

# Prediction 수행

###  Raw Text를 BERT Tokens로 변환하기 위한 Request Handler 설정

In [ ]:
from sagemaker.serializers import JSONSerializer

class RequestHandler(JSONSerializer):
    import json
    
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.content_type = "application/json"

    def serialize(self, instances):
        transformed_instances = []

        for instance in instances:
            encode_plus_tokens = tokenizer.encode_plus(instance,
                                                       pad_to_max_length=True,
                                                       max_length=self.max_seq_length)

            input_ids = encode_plus_tokens['input_ids']
            input_mask = encode_plus_tokens['attention_mask']
            segment_ids = [0] * self.max_seq_length

            transformed_instance = {"input_ids": input_ids, 
                                    "input_mask": input_mask, 
                                    "segment_ids": segment_ids}

            transformed_instances.append(transformed_instance)

        transformed_data = {"instances": transformed_instances}

        return json.dumps(transformed_data)

###  BERT Response를 Predicted Classes로 변환하기 위한 Response Handler 설정

In [ ]:
from sagemaker.deserializers import JSONDeserializer

class ResponseHandler(JSONDeserializer):
    import json
    import tensorflow as tf
    
    def __init__(self, classes):
        self.classes = classes
        self.accept = 'application/json'
        
    def deserialize(self, response, accept_header):
        import tensorflow as tf

        response_body = response.read().decode('utf-8')

        response_json = json.loads(response_body)

        log_probabilities = response_json["predictions"]

        predicted_classes = []

        # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
        for log_probability in log_probabilities:
            softmax = tf.nn.softmax(log_probability)    
            predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
            predicted_class = self.classes[predicted_class_idx]
            predicted_classes.append(predicted_class)

        return predicted_classes

In [ ]:
import json
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

request_handler = RequestHandler(tokenizer=tokenizer,
                                 max_seq_length=128)

response_handler = ResponseHandler(classes=[1, 2, 3, 4, 5])

## Predictor 객체 선언

In [ ]:
from sagemaker.tensorflow.model import TensorFlowPredictor

predictor = TensorFlowPredictor(
                        endpoint_name=endpoint_name,
                        sagemaker_session=sess,
                        serializer=request_handler,
                        deserializer=response_handler,
                        model_name='saved_model',
                        model_version=0)

In [ ]:
# import tensorflow as tf
# import json
    
reviews = ["This is great!", 
           "This is not good."]

predicted_classes = predictor.predict(reviews)
%timeit predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

# REST Endpoint 성능 지표 검토

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint Performance Metrics</a></b>'.format(region, endpoint_name)))


# Variant A로 모든 트래픽 이동


_**No downtime** 트래픽 이동을 위한 작업이 수행되는 동안에 downtime이 없습니다._

이 작업에도 몇 분의 시간이 걸립니다.

In [ ]:
updated_endpoint_config = [
    {
        'VariantName': 'VariantA',
        'DesiredWeight': 100,
    },
    {
        'VariantName': 'VariantB',
        'DesiredWeight': 0,
    }
]

In [ ]:
client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name,
    DesiredWeightsAndCapacities=updated_endpoint_config
)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint</a></b>'.format(region, endpoint_name)))


<h2><span style="color:red">위 Endpoint가 update되는 동안은 기다려 주시기 바랍니다.</span></h2>

In [ ]:
client = boto3.client('sagemaker')
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

In [ ]:
# import tensorflow as tf
# import json
    
reviews = ["This is great!", 
           "This is not good."]

predicted_classes = predictor.predict(reviews)
%timeit predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

# 비용절감을 위한 Variant A 삭제

Endpoint configuration은 variant B만 사용하도록 수정합니다.

_**No downtime** 트래픽 이동을 위한 작업이 수행되는 동안에 downtime이 없습니다._

이 작업에도 몇 분의 시간이 걸립니다.

In [ ]:
import time
timestamp = '{}'.format(int(time.time()))

updated_endpoint_config_name = '{}-{}'.format(training_job_name, timestamp)

updated_endpoint_config = client.create_endpoint_config(
    EndpointConfigName=updated_endpoint_config_name,
    ProductionVariants=[
        {
         'VariantName': 'VariantB',
         'ModelName': model_b_name,  # Only specify variant B to remove variant A
         'InstanceType':'ml.m5.large',
         'InitialInstanceCount': 1,
         'InitialVariantWeight': 100
        }
    ])

In [ ]:
client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=updated_endpoint_config_name
)

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint</a></b>'.format(region, endpoint_name)))


In [ ]:
client = boto3.client('sagemaker')
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

In [ ]:
# import tensorflow as tf
# import json
    
reviews = ["This is great!", 
           "This is not good."]

predicted_classes = predictor.predict(reviews)
%timeit predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

In [ ]:
model_ab_endpoint = endpoint_name

In [ ]:
%store model_ab_endpoint

### Delete Endpoint


In [ ]:
# list_endpoints = sm.list_endpoints()

# for ep in list_endpoints['Endpoints']:
#     sm.delete_endpoint(EndpointName=ep['EndpointName'])
    

# NextToken = 'None'
# while NextToken !='':
#     lec = sm.list_endpoint_configs(NextToken=NextToken) if NextToken != 'None' else sm.list_endpoint_configs()
#     for epc in lec['EndpointConfigs']:
#         print(epc['EndpointConfigName'])
#         sm.delete_endpoint_config(EndpointConfigName=epc['EndpointConfigName'])
#         time.sleep(3)
#     NextToken = lec['NextToken'] if lec.get('NextToken') else ''

# NextToken = 'None'
# while NextToken !='':
#     lec = sm.list_models(NextToken=NextToken) if NextToken != 'None' else sm.list_models()
#     for epc in lec['Models']:
#         print(epc['ModelName'])
#         sm.delete_model(ModelName=epc['ModelName'])
#         time.sleep(3)
#     NextToken = lec['NextToken'] if lec.get('NextToken') else ''